In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data/data11.csv')
print(data)
# conversion of dataframe to list
points = data.to_numpy().tolist()
print(points)

   x  y
0  1  1
1  3  2
2  9  1
3  3  7
4  7  2
5  9  7
6  4  8
7  8  3
8  1  4
[[1, 1], [3, 2], [9, 1], [3, 7], [7, 2], [9, 7], [4, 8], [8, 3], [1, 4]]


In [2]:
# distance metrics
def euclid(x, y):
    return ((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2) ** 0.5

def manhatten(x, y):
    return abs(x[0] - y[0]) + abs(x[1] - y[1])

def minkowsky(x, y, p):
    return (((x[0] - y[0]) ** p) + ((x[1] - y[1]) ** p)) ** (1 / p)

In [3]:
# distance matrix creation
# p will be used if metric is minkowsky
def matrix_creation(points, metric, p = 2):
    n = len(points)
    mat = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if metric == 'euclid':
                mat[i][j] = euclid(points[i], points[j])
            if metric == 'manhatten':
                mat[i][j] = manhatten(points[i], points[j])
            else:
                mat[i][j] = minkowsky(points[i], points[j], p)
    return mat

In [4]:
# heirarchal clustering metrics
def single(mat, i, j, store):
    return min(mat[x][y] for x in store[i] for y in store[j])
    
def complete(mat, i, j, store):
    return max(mat[x][y] for x in store[i] for y in store[j])
    
def average(mat, i, j, store):
    ret = 0
    for x in store[i]:
        for y in store[j]:
            ret += mat[x][y]
    return ret / (len(store[i]) * len(store[j]))

def centroid(points, i, j, store):
    x = np.array(points)
    a = np.mean(x[store[i]], axis = 0)
    b = np.mean(x[store[j]], axis = 0)
    return euclid(a, b)

def ward(points, i, j, store):
    x = np.array(points)
    centroid_i = np.mean(x[store[i]], axis = 0)
    centroid_j = np.mean(x[store[j]], axis = 0)
    a, b = len(store[i]), len(store[j])
    return ((a * b) / (a + b)) * (euclid(centroid_i, centroid_j) ** 2)

In [5]:
# heirarchal clustering starts

def heirarchal_clustering(points, metric_dis, metric_cluster, p = 2):
    # store is used to track all current clusters
    store = [[i] for i in range(len(points))]
    # get distance matrix
    mat = matrix_creation(points, metric_dis, p)
    
    while len(store) > 1:
        
        print(store)
        
        merge = (-1, -1)
        
        if metric_cluster == 'single':
            dist = float('inf')
            for i in range(len(store)):
                for j in range(i + 1, len(store)):
                    val = single(mat, i, j, store)
                    if val < dist:
                        dist = val
                        merge = (i, j)
                        
        elif metric_cluster == 'complete':
            dist = float('inf')
            for i in range(len(store)):
                for j in range(i + 1, len(store)):
                    val = complete(mat, i, j, store)
                    if val < dist:
                        dist = val
                        merge = (i, j)
                        
        elif metric_cluster == 'average':
            dist = float('inf')
            for i in range(len(store)):
                for j in range(i + 1, len(store)):
                    val = average(mat, i, j, store)
                    if val < dist:
                        dist = val
                        merge = (i, j)
                        
        elif metric_cluster == 'centroid':
            dist = float('inf')
            for i in range(len(store)):
                for j in range(i + 1, len(store)):
                    val = centroid(points, i, j, store)
                    if val < dist:
                        dist = val
                        merge = (i, j)

        elif metric_cluster == 'ward':
            dist = float('inf')
            for i in range(len(store)):
                for j in range(i + 1, len(store)):
                    val = ward(points, i, j, store)
                    if val <= dist:
                        dist = val
                        merge = (i, j)

        # finding clusters that have to be merged
        a = store[merge[0]]
        b = store[merge[1]]
        # merging clusters
        store = store[: merge[0]] + store[merge[0] + 1: merge[1]] + store[merge[1] + 1: ] + [a + b]
        
    print(store)

heirarchal_clustering(points, 'euclid', 'ward')
        

[[0], [1], [2], [3], [4], [5], [6], [7], [8]]
[[0], [1], [2], [3], [5], [6], [8], [4, 7]]
[[0], [1], [2], [5], [8], [4, 7], [3, 6]]
[[2], [5], [8], [4, 7], [3, 6], [0, 1]]
[[5], [8], [3, 6], [0, 1], [2, 4, 7]]
[[5], [3, 6], [2, 4, 7], [8, 0, 1]]
[[3, 6], [8, 0, 1], [5, 2, 4, 7]]
[[5, 2, 4, 7], [3, 6, 8, 0, 1]]
[[5, 2, 4, 7, 3, 6, 8, 0, 1]]
